In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Kidney_Disesae_Classification_with-MLFLOW_-_DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Kidney_Disesae_Classification_with-MLFLOW_-_DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen= True)
class DataIngestionConfig:
    root_dir : Path
    source_url: str
    local_data_path: Path
    unzip_data_path: Path

In [7]:
from src.Kidney_Disease_Classification.constants import *
from src.Kidney_Disease_Classification.utils.common import read_yaml, create_directory

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir])


        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_path = config.local_data_path,
            unzip_data_path = config.unzip_data_path

        )
        return data_ingestion_config

In [9]:
import os
import zipfile
import gdown
from src.Kidney_Disease_Classification import logger
from src.Kidney_Disease_Classification.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config


    def download_file(self) ->str:
        """Fetch data from url"""

        try:
            dataset_url = self.config.source_url
            zipdata_path = self.config.local_data_path

            os.makedirs("artifacts/data_ingestion", exist_ok=True)

            logger.info(f"Downloading is started")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='

            gdown.download(prefix+file_id, zipdata_path)

            logger.info("Downloaded data from url")
            
        except Exception as e:
            logger.info(e)
            raise e
        

    def extract_zip_file(self):
        """ This method is responsible for unziping the data"""

        unzip_dir = self.config.unzip_data_path
        os.makedirs(unzip_dir, exist_ok= True)

        with zipfile.ZipFile(self.config.local_data_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

            


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-15 12:38:53,489  - INFO - common - Reading done from config\config.yaml file]
[2023-11-15 12:38:53,497  - INFO - common - Reading done from params.yaml file]
[2023-11-15 12:38:53,500  - INFO - common - Directory is created for artifacts]
[2023-11-15 12:38:53,501  - INFO - common - Directory is created for artifacts/data_ingestion]
[2023-11-15 12:38:53,503  - INFO - 1388353949 - Downloading is started]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1n1WYibOlWcTc-coQ8mPcHipvxWMfir0j
From (redirected): https://drive.google.com/uc?/export=download&id=1n1WYibOlWcTc-coQ8mPcHipvxWMfir0j&confirm=t&uuid=5aa5c807-37e1-4683-b72e-844d72619b5c
To: d:\Projects\Kidney_Disesae_Classification_with-MLFLOW_-_DVC\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:05<00:00, 10.1MB/s]

[2023-11-15 12:39:00,540  - INFO - 1388353949 - Downloaded data from url]
